In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import math
import time

def load_tsp(file_path):
    # 加載 TSP 節點
    with open(file_path, 'r') as f:
        lines = f.readlines()

    nodes = []
    start_reading = False
    for line in lines:
        if start_reading:
            parts = line.strip().split()
            if len(parts) == 3:
                nodes.append((float(parts[1]), float(parts[2])))
        if line.strip() == "NODE_COORD_SECTION":
            start_reading = True
        if line.strip() == "EOF":
            break

    return nodes

def calculate_distances(nodes):
    # 計算節點間的距離矩陣
    num_nodes = len(nodes)
    distances = np.zeros((num_nodes, num_nodes))
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distances[i][j] = math.sqrt((nodes[i][0] - nodes[j][0])**2 + (nodes[i][1] - nodes[j][1])**2)
    return distances

def evaluate_solution(solution, distances=None):
    # 計算目標函數值
    total_distance = sum(distances[solution[i]][solution[i + 1]] for i in range(len(solution) - 1))
    total_distance += distances[solution[-1]][solution[0]]  # 回到起點
    return total_distance

def initialize_solution(num_variables):
    # 初始化解
    return random.sample(range(num_variables), num_variables)

def perform_attack(solution, best_solution, distances=None, max_attempts=6):
    # 攻擊策略(隨機交換或隨機調整)
    new_solution = solution[:]
    for _ in range(max_attempts):
        index1, index2 = random.sample(range(len(solution)), 2)
        new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]

        if evaluate_solution(new_solution, distances) < evaluate_solution(best_solution, distances):
            return new_solution
    return best_solution

def perform_defense(solution, temperature, distances=None, max_attempts=5):
    # 防守策略(模擬退火)
    new_solution = solution[:]
    for _ in range(max_attempts):
        index1, index2 = random.sample(range(len(solution)), 2)
        new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]

        current_score = evaluate_solution(solution, distances)
        new_score = evaluate_solution(new_solution, distances)
        if new_score < current_score or random.uniform(0, 1) < math.exp((current_score - new_score) / temperature):
            return new_solution
    return solution

def football_algorithm(file_path, num_variables=None, total_iterations=2000, population_size=50, initial_temperature=100, cooling_rate=0.99):
    # 足球演算法:
    nodes = load_tsp(file_path)
    distances = calculate_distances(nodes)
    num_variables = len(nodes)

    population = [initialize_solution(num_variables) for _ in range(population_size)]
    best_solution = min(population, key=lambda sol: evaluate_solution(sol, distances))
    best_score = evaluate_solution(best_solution, distances)
    temperature = initial_temperature

    start_time = time.time()
    scores = []

    for iteration in range(total_iterations):
        for i in range(population_size):
            if random.random() < 0.5:
                population[i] = perform_attack(population[i], best_solution, distances)
            else:
                population[i] = perform_defense(population[i], temperature, distances)

        temperature *= cooling_rate

        # 更新最佳解
        current_best = min(population, key=lambda sol: evaluate_solution(sol, distances))
        current_score = evaluate_solution(current_best, distances)
        if current_score < best_score:
            best_solution = current_best
            best_score = current_score

        scores.append(best_score)

    end_time = time.time()

    # 繪製收斂圖
    plt.figure(figsize=(10, 6))
    plt.plot(scores, label='Best Score')
    plt.title('TSP')
    plt.xlabel('Iteration')
    plt.ylabel('Objective Value')
    plt.legend()
    plt.grid()
    plt.show()

    print(f"最佳解: {best_solution}")
    print(f"最佳目標值: {best_score:.2f}")
    print(f"總計算時間: {end_time - start_time:.2f} 秒")

    return best_solution, best_score, end_time - start_time

# 執行 TSP 問題解決
tsp_solution, tsp_score, tsp_time = football_algorithm(file_path='pr144.tsp')